-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

# Sistemas de recomendación

## ¿Qué son las recomendaciones?
¿Cómo hace YouTube para saber que video quisieras ver después de que se acabe el que estás viendo? o ¿Cómo elige Google Play Store una aplicación relevante para ti? ¿Magia? No, en ambos casos un modelo de recomendación basado en Aprendizaje de Maquinas determina qué tan similares son unos videos o series con otras cosas que ya has visto y por ende sirven para crear recomendaciones. Generalmente, las aplicaciones usan dos tipos de recomendaciones:

- recomendaciones de la página de inicio.
- recomendaciones de artículos relacionados.

**Recomendaciones de página de inicio**

Las recomendaciones de la página de inicio se personalizan para un usuario en función de sus intereses conocidos. Cada usuario ve diferentes recomendaciones.

Si va a la página de inicio de Google Play Apps, puede ver algo como esto:

<center>
<img src="https://developers.google.com/machine-learning/recommendation/images/PlayStore.svg" alt="play_store_home" width="200" height="300">
</center>

**Recomendaciones de artículos relacionados**

Como sugiere el nombre, los artículos relacionados son recomendaciones similares a un artículo en particular. En el ejemplo de las aplicaciones de Google Play, los usuarios que buscan una página para una aplicación de matemáticas también pueden ver un panel de aplicaciones relacionadas, como otras aplicaciones de matemáticas o ciencias.

**¿Por qué son importantes las recomendaciones?**

Un sistema de recomendación ayuda a los usuarios a encontrar contenido atractivo en un gran corpus. Por ejemplo, Google Play Store ofrece millones de aplicaciones, mientras que YouTube ofrece miles de millones de videos. Cada día se agregan más aplicaciones y videos. ¿Cómo pueden los usuarios encontrar nuevo contenido atractivo? Sí, uno puede usar la búsqueda para acceder al contenido. Sin embargo, un motor de recomendaciones puede mostrar elementos que los usuarios podrían no haber pensado en buscar por su cuenta.

## Terminología
Antes de adentrarnos en los algoritmos, hay algunos términos que debe conocer:

**Elementos o items (también son conocidos como documentos)**

Son las entidades que recomienda un sistema. Para la tienda Google Play, los elementos son aplicaciones para instalar. Para YouTube, los elementos son videos.

**Consulta o query (también son conocidos como contexto)**

La información que utiliza un sistema para hacer recomendaciones. Las consultas pueden ser una combinación de lo siguiente:

- Informacion del usuario:
    - La identificación del usuario,
    - Elementos con los que los usuarios interactuaron previamente, etc
- Contexto adicional
    - Hora del día,
    - El dispositivo del usuario, etc

**Embedding**

Es un mapeo de un conjunto discreto (en este caso, el conjunto de consultas o el conjunto de elementos a recomendar) a un espacio vectorial llamado espacio incrustado (_embedding space_). Muchos sistemas de recomendación se basan en el aprendizaje de una representación incrustada adecuada de las consultas y los elementos.

## Descripción general de los sistemas de recomendación
Una arquitectura común para los sistemas de recomendación consta de los siguientes componentes:

- Generación de candidatos,
- Puntuación y
- Reclasificación.

**Generación de candidatos**

En esta primera etapa, el sistema parte de un corpus potencialmente enorme y genera un subconjunto mucho más pequeño de candidatos. Por ejemplo, el generador de candidatos en YouTube reduce miles de millones de videos a cientos o miles. El modelo necesita evaluar consultas rápidamente dado el enorme tamaño del corpus. Un modelo dado puede proporcionar múltiples generadores de candidatos, cada uno de los cuales nomina un subconjunto diferente de candidatos.

**Puntuación**

A continuación, otro modelo puntúa y clasifica a los candidatos para seleccionar el conjunto de elementos (del orden de 10) para mostrar al usuario. Dado que este modelo evalúa un subconjunto relativamente pequeño de elementos, el sistema puede utilizar un modelo más preciso basándose en consultas adicionales.

**Reclasificación**

Finalmente, el sistema debe tener en cuenta restricciones adicionales para la clasificación final. Por ejemplo, el sistema elimina elementos que al usuario no le gustaron explícitamente o aumenta la puntuación de contenido más nuevo. La reclasificación también puede ayudar a garantizar la diversidad, la frescura y la equidad.

Discutiremos cada una de estas etapas a lo largo del notebook y daremos ejemplos de diferentes sistemas de recomendación, como YouTube.

Si desea entrar a profundidad a conocer los detalles de cómo funciona la arquitectura y el algoritmo de YouTube, puede revisar el siguiente paper:

Covington, P., Adams, J., &; Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190 

### Descripción general de la generación de candidatos
La generación de candidatos es la primera etapa de la recomendación. Ante una consulta, el sistema genera un conjunto de candidatos relevantes. La siguiente tabla muestra dos enfoques comunes de generación de candidatos:

|             **Tipo**             |                                              **Definición**                                              |                                                                                                   **Ejemplo**                                                                                                   |
|:--------------------------------:|:--------------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| **Filtrado basado en contenido** | Utiliza la _similitud entre elementos_ para recomendar elementos similares a los que le gustan al usuario. |                                           Si el usuario A ve dos videos de gatos tiernos, entonces el sistema puede recomendar videos de animales tiernos a ese usuario.                                          |
|    **Filtración colaborativa**   |    Utiliza _similitudes entre consultas y elementos simultáneamente_ para proporcionar recomendaciones.    | Si el usuario A es similar al usuario B y al usuario B le gusta el video 1, entonces el sistema puede recomendar el video 1 al usuario A (incluso si el usuario A no ha visto ningún video similar al video 1). |

**Espacio de incrustación (Embedding space)**

Tanto el filtrado colaborativo como el basado en contenido asignan cada elemento y cada consulta (o contexto) a un vector de incrustación en un espacio de incrustación común $E=\mathbb{R}^d$. Por lo general, el espacio de incrustación es de baja dimensión (es decir, es mucho más pequeño que el tamaño del corpus) y captura alguna estructura latente del elemento o conjunto de consultas. Los elementos similares, como los videos de YouTube que suele ver el mismo usuario, terminan juntos en el espacio de incrustación. La noción de "cercanía" se define por una medida de similitud.

**Medidas de similitud**

Una medida de similitud es una función $s: E\times E \rightarrow \mathbb{R}$ que toma un par de incrustaciones y devuelve un escalar que mide su similitud. Las incrustaciones se pueden utilizar para la generación de candidatos de la siguiente manera: dada una incrustación de consulta $q\in E$, el sistema busca incrustaciones de elementos $x\in E$ que estén cerca de $q$, es decir, incrustaciones con una gran similitud $s(q, x)$.

Para determinar el grado de similitud, la mayoría de los sistemas de recomendación se basan en uno o más de los siguientes:
- Coseno
- Producto punto
- Distancia euclidiana

**Coseno**

Es simplemente el coseno del angulo entre dos vectores: $s(q,x)=cos(q,x)$

**Producto punto**

El producto punto entre dos vectores es $s(q,x)=\langle q,x \rangle=\sum_{i=1}^d q_i x_i$. A su vez esta expresión es igual a $s(q,x)=\lVert x\rVert \lVert q \rVert cos(q,x)$ (el coseno del ángulo multiplicado por el producto de las normas). Por lo tanto, si las incrustaciones se normalizan, el producto escalar y el coseno coinciden.

**Distancia euclideana**

Esta es la distancia habitual en el espacio euclidiano, $s(q,x)=\lVert q - x \rVert=\left[ \sum_{i=1}^d (q_i - x_i)^2 \right]^{\frac{1}{2}}$. Una distancia más pequeña significa una mayor similitud. Tenga en cuenta que cuando se normalizan las incrustaciones, la distancia euclidiana al cuadrado coincide con el producto punto (y el coseno) ya que se puede demostrar que: $\frac{1}{2}\lVert q - x \rVert^2=1-\langle q,x \rangle$

### ¿Qué medida de similitud elegir?
En comparación con el coseno, la similitud del producto escalar es sensible a la norma de la incrustación. Es decir, cuanto mayor sea la norma de una incrustación, mayor será la similitud (para elementos con un ángulo agudo) y más probable es que se recomiende el elemento. Esto puede afectar las recomendaciones de la siguiente manera:
- Los elementos que aparecen con mucha frecuencia en el conjunto de entrenamiento (por ejemplo, videos populares de YouTube) tienden a tener incrustaciones con normas grandes. Si desea capturar información de popularidad, debe preferir el producto punto. Sin embargo, si no tiene cuidado, los artículos populares pueden terminar dominando las recomendaciones. En la práctica, puede utilizar otras variantes de medidas de similitud que pongan menos énfasis en la norma del elemento. Por ejemplo, definir $s(q,x)=\lVert q \rVert^\alpha \lVert x \rVert^\alpha cos(q,x)$ para algún $\alpha\in(0,1)$
- Es posible que los elementos que aparecen muy raramente no se actualicen con frecuencia durante el entrenamiento. En consecuencia, si se inicializan con una norma grande, el sistema puede recomendar artículos raros sobre artículos más relevantes. Para evitar este problema, tenga cuidado al incrustar la inicialización y utilice la regularización adecuada.


## Filtrado basado en contenido


Este utiliza las características de los elementos para recomendar otros elementos similares a los que ya le gustan al usuario, en función de sus acciones anteriores o comentarios explícitos.

A modo ilustrativo, presentaremos la siguiente matriz en donde cada fila es una aplicación y cada columna representa una característica de la aplicación. Si una entrada de la matriz es 0 significa que la aplicación de esa fila no cuenta con la característica de esa columna. En caso de que la entrada sea 1, significa que la aplicación cuenta con la característica.

A su vez, podemos representar al usuario en el mismo espacio, por ende, podríamos describir sus preferencias como un vector fila donde cada entrada hace alusión a alguna característica de la aplicación. Que la entrada sea 1 o 0 dependerá de la información explicita que haya del usuario como preferencias declaradas en su bio o implícita a partir de su comportamiento de consumo. 

<center>
<img src = "data/sistemas_recomendacion.png" alt = "preferencias" style = "width: 500px;"/>
</center>

De la matriz anterior note que las primeras tres filas corresponden a tres aplicaciones. Las columnas representan atributos de las aplicaciones o las preferencias. Las primeras tres columnas se refieren a la temática de la aplicación, las últimas tres muestran la empresa que desarrolló la aplicación. A su vez, la última fila muestra a nuestro usuario, Séneca. Este tiene gustos por las aplicaciones de diversión y productividad. Del mismo modo, cuenta con alguna aplicación de la Empresa A en su celular.

El objetivo del modelo es recomendar elementos (en este caso aplicaciones) relevantes para el usuario. Para hacer eso primero debe construir una métrica de similitud (por ejemplo el producto punto). Luego, debe crear el sistema para otorgarle una puntuación a cada aplicación candidata de acuerdo a la medida de similitud. Note que las recomendaciones son especificas al usuario. En ningún momento se está utilizando información de otros usuarios.

En este caso vamos a utilizar el Producto Punto como medida de similitud. Supongamos que el embedimiento del usuario Séneca es $x$ y el embedimiento de la aplicación es $y$, ambos son vectores binarios. Entonces $\langle x,y \rangle=\sum_{i=1}^dx_iy_i$ cuando un atributo aparece en ambos vectores, este contribuye 1 a la suma. En otras palabras, $\langle x,y \rangle$ es el número de características que están activas en los dos vectores simultáneamente. Por ende, un Producto Punto alto corresponde a una alta similaridad entre objetos.

En ese orden de ideas, la aplicación creada por la Empresa A es la más cercana a los gustos de Séneca pues comparten dos características en común. Las otras dos aplicaciones tienen un producto punto de 1 con Séneca, por ende son un poco más disimiles a los gustos del usuario.

**Ventajas**
- El modelo no necesita ningún dato sobre otros usuarios, ya que las recomendaciones son específicas para este usuario. Esto hace que sea más fácil de escalar a una gran cantidad de usuarios.
- El modelo puede capturar los intereses específicos de un usuario y puede recomendar elementos de nicho en los que muy pocos usuarios están interesados.

**Desventajas** 
- El modelo solo puede hacer recomendaciones basadas en los intereses existentes del usuario. En otras palabras, el modelo tiene una capacidad limitada para expandir los intereses existentes de los usuarios.
- Requiere bastante conocimiento sobre el negocio para que los atributos que generan la recomendación sean relevantes.

## Filtración colaborativa

Para solventar una de las desventajas del filtrado basado en contenido se puede usar la técnica de Filtración Colaborativa. Esta utiliza similitudes entre usuarios y elementos de forma simultanea para crear las recomendaciones. Por ende, esto nos permite recomendarle elementos al individuo A a partir de los gustos del individuo B. Adicionalmente, estos embedimientos se pueden entrenar para que aprendan de forma automática sin necesidad de realizar ingeniería manual de características.

**Ejemplo de recomendación de una película**
Considere un sistema de recomendación de películas en el que los datos de entrenamiento consisten en una matriz de retroalimentación en la que cada fila representa un usuario y cada columna representa un elemento (una película).

Los comentarios sobre las películas se dividen en una de dos categorías:
- Explícito: los usuarios especifican cuánto les gustó una película en particular proporcionando una calificación numérica.
- Implícito: si un usuario mira una película, el sistema infiere que el usuario está interesado.

Para simplificar, supondremos que la matriz de retroalimentación es binaria; es decir, un valor de 1 indica interés en la película.

Cuando un usuario visita la página de inicio, el sistema debería recomendar películas basándose tanto en la similitud con películas que le han gustado al usuario en el pasado y con películas que les gustaron a usuarios similares.

A modo de ilustración traemos la siguiente tabla:


|                **Película**               | **Clasificación** |                                                                                                  **Descripción**                                                                                                  |
|:-----------------------------------------:|:-----------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Batman: El caballero de la noche asciende |       PG-13       |                              Batman se esfuerza por salvar a Gotham City de la aniquilación nuclear en esta secuela del Caballero de la Noche ambientada en el universo de DC Comics.                             |
|     Harry Potter y la Piedra Filosofal    |         PG        |                       Un niño huérfano descubre que es un mago y se matricula en el Colegio Hogwarts de Magia y Hechicería donde libra su primera batalla contra el malvado Lord Voldemort.                       |
|                   Shrek                   |         PG        |                                Un ogro adorable y su compañero burro se embarcan en una misión para rescatar a la princesa Fiona que está encarcelada en su castillo por un dragón.                               |
|        Las trillizas de Belleville        |       PG-13       | Cuando el ciclista profesional Champion es secuestrado durante el Tour de Francia, su abuela y su perro con sobrepeso viajan al extranjero para rescatarlo con la ayuda de un trío de ancianos cantantes de jazz. |
|                  Memento                  |         R         |                                                       Un amnésico busca desesperadamente resolver el asesinato de su esposa tatuándose pistas en su cuerpo.                                                       |

### Incrustación o embebimiento 1D

Suponga que asignamos a cada película un escalar en el dominio $[-1,1]$ que describe si la película es para niños (valores negativos) o para adultos (valores positivos). Supongamos que también asignamos un escalar a cada usuario que describe el interés del usuario en películas para niños (más cercano a -1) o películas para adultos (más cercano a +1). El producto de la incrustación de la película y la incrustación del usuario debe ser más alto (más cercano a 1) para las películas que esperamos que le gusten al usuario.

<center>
<img src = "data/embedding-1D.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

A continuación se va a representar en una matriz las películas que cada usuario ha visto. Los usuarios están representados en las filas y las películas en las columnas. En particular note que el tercer y cuarto usuario tienen preferencias que están bien explicadas por esta función: el tercer usuario prefiere películas para niños y el cuarto usuario prefiere películas para adultos. Sin embargo, las preferencias del primer y segundo usuario no se explican bien por esta única característica.

<center>
<img src = "data/embedding-matriz.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

Note que la clasificación de las películas según su público está dado y que la clasificación de los usuarios se construye a partir de las películas que ha visto y su clasificación. Decimos que las preferencias del usuario 3 y 4 están bien descritas pues estas toman valores extremos por lo que claramente sabemos qué tipo de películas ve cada uno.

### Incrustación o embebimiento 2D
Una característica no fue suficiente para explicar las preferencias de todos los usuarios. Para superar este problema, agreguemos una segunda característica: el grado en que cada película es un éxito de taquilla o una película independiente y de culto. Con una segunda función, ahora podemos representar cada película con la siguiente incrustación bidimensional:

<center>
<img src = "data/embedding-2D.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

En este ejemplo, diseñamos a mano las incrustaciones. En la práctica, las incrustaciones se pueden aprender automáticamente, que es el poder de los modelos de filtrado colaborativo. A continuación aprenderemos sobre diferentes modelos para aprender estas incrustaciones.

La naturaleza colaborativa de este enfoque es evidente cuando el modelo aprende las incrustaciones. Supongamos que los vectores de incrustación de las películas son fijos. Luego, el modelo puede aprender un vector de incrustación para que los usuarios expliquen mejor sus preferencias. En consecuencia, las incorporaciones de usuarios con preferencias similares estarán muy juntas. De manera similar, si las incrustaciones para los usuarios son fijas, entonces podemos aprender incrustaciones de películas para explicar mejor la matriz de visualización de películas y usuarios. Como resultado, las incrustaciones de películas que gustan a usuarios similares estarán cerca en el espacio de incrustación.

## Factorización de matrices
La factorización de matrices es un modelo de incrustación simple. Dada la matriz de retroalimentación $A\in\mathbb{R}^{ m\times n}$ donde $m$ es el número de usuarios (o consultas) y $n$ es el número de elementos, el modelo aprende:
- Una matriz de incrustación de usuario $U\in\mathbb{R}^{ m\times d}$ donde la fila $i$ es la incrustación del usuario $i$.
- Una matriz de incrustación de elementos $V\in\mathbb{R}^{ n\times d}$, donde la fila $j$ es la incrustación del elemento $j$.

<center>
<img src = "data/matrix_factorization.png" alt = "UVT" style = "width: 500px;"/>
</center>

Las incrustaciones se aprenden de manera que el producto $UV^T$ sea una buena aproximación de la matriz de retroalimentación $A$. Observe que la entrada $(i,j)$ de $UV^T$ es simplemente el producto escalar $\langle U_i, V_j \rangle$ de las incrustaciones del usuario $i$ y el elemento $j$, que busca aproximar $A_{i,j}$.

La factorización de matrices generalmente brinda una representación más compacta que aprender la matriz completa. La matriz completa tiene $O(nm)$ entradas, mientras que las matrices incrustadas $U, V$ tienen $O((n+m)d)$ entradas, donde la dimensión incrustada $d$ suele ser mucho más pequeña que $m$ y $n$. Como resultado, la factorización de matrices encuentra una estructura latente en los datos, asumiendo que las observaciones se encuentran cerca de un subespacio de baja dimensión. En el ejemplo anterior, los valores de $n$, $m$ y $d$ son tan bajos que la ventaja es insignificante. Sin embargo, en los sistemas de recomendación del mundo real, la factorización de matrices puede ser significativamente más compacta que aprender la matriz completa.

### Elección de la función objetivo
Una función objetivo intuitiva es la distancia al cuadrado. Para hacer esto se minimiza la suma de los errores cuadráticos sobre todos los pares de entradas observadas:
$$\min_{U\in\mathbb{R}^{m\times d}, V\in\mathbb{R}^{n\times d}} \sum_{(i,j)\in obs}(A_{i,j} - \langle U_i, V_j \rangle)^2$$

En esta función objetivo, solo se suma sobre los pares observados $(i, j)$, es decir, sobre valores distintos de cero en la matriz de retroalimentación. Sin embargo, solo sumar los valores de uno no es una buena idea: una matriz de todos los unos tendrá una pérdida mínima y producirá un modelo que no puede hacer recomendaciones efectivas y que se generaliza mal.

<center>
<img src = "data/MF.png" alt = "MF" style = "width: 500px;"/>
</center>

Tal vez podría tratar los valores no observados como cero y sumar todas las entradas en la matriz. Esto corresponde a minimizar la distancia de [Frobenius](https://es.wikipedia.org/wiki/Norma_matricial#Norma_de_Frobenius) al cuadrado entre $A$ y su aproximación $UV^T$:

$$\min_{U\in\mathbb{R}^{m\times d}, V\in\mathbb{R}^{n\times d}} ||A-UV^T||^2_F$$

Puede resolver este problema cuadrático a través de la **descomposición en valores singulares (SVD)** de la matriz. Sin embargo, SVD tampoco es una gran solución, porque en aplicaciones reales, la matriz puede ser muy sparse (llena de ceros). Por ejemplo, piense en todos los videos de YouTube en comparación con todos los videos que ha visto un usuario en particular. La solución $UV^T$ (que corresponde a la aproximación del modelo de la matriz de entrada) probablemente será cercana a cero, lo que conducirá a un rendimiento de generalización deficiente.

Por el contrario, la **factorización matricial ponderada** descompone el objetivo en las siguientes dos sumas:
- Una suma sobre las entradas observadas.
- Una suma sobre entradas no observadas (tratadas como ceros).

$$\min_{U\in\mathbb{R}^{m\times d}, V\in\mathbb{R}^{n\times d}} \sum_{(i,j)\in obs}(A_{i,j} - \langle U_i, V_j \rangle)^2 + \omega_0 \sum_{(i,j)\not\in obs} (\langle U_i, V_j \rangle)^2$$

Aquí, $\omega_0$ es un hiperparámetro que pondera los dos términos para que el objetivo no esté dominado por uno u otro. Ajustar este hiperparámetro es muy importante.

En aplicaciones prácticas, también debe ponderar cuidadosamente los pares observados. Por ejemplo, los elementos frecuentes (por ejemplo, videos de YouTube extremadamente populares) o consultas frecuentes (por ejemplo, usuarios frecuentes) pueden dominar la función objetivo. Puede corregir este efecto ponderando las observaciones de la base de entrenamiento para tener en cuenta la frecuencia de los elementos. En otras palabras, puede reemplazar la función objetivo por:

$$\sum_{(i,j)\in obs}\omega_{i,j} (A_{i,j} - \langle U_i, V_j \rangle)^2 + \omega_0 \sum_{(i,j)\not\in obs} (\langle U_i, V_j \rangle)^2$$

En donde $\omega_{i,j}$ es una función de la frecuencia de la consulta $i$ y del elemento $j$.

### Minimizar la función objetivo
Entre los algoritmos más comunes para minimizar una función objetivo están:

- El descenso de gradiente estocástico (SGD) es un método genérico para minimizar las funciones de pérdida. En este algoritmo de descenso de gradiente el que el tamaño del lote (*batch size*) es uno. En otras palabras, SGD se basa en un solo ejemplo elegido uniformemente al azar de un conjunto de datos para calcular una estimación del gradiente en cada paso. Los métodos de descenso por gradiente ajustan iterativamente los parámetros del modelo encontrando gradualmente la mejor combinación de pesos para minimizar la pérdida.
- Los mínimos cuadrados alternos ponderados (WALS).

El objetivo es cuadrático en cada una de las dos matrices $U$ y $V$. (Tenga en cuenta, sin embargo, que el problema no es conjuntamente convexo). WALS funciona inicializando las incrustaciones aleatoriamente y luego alternando entre:
- Dejando fija $U$ y resolviendo para $V$.
- Dejando fija $V$ y resolviendo para $U$.

Cada etapa se puede resolver exactamente (a través de la solución de un sistema lineal) y se puede distribuir computacionalmente. Se puede demostrar que esta técnica convergerá porque se garantiza que con cada iteración se disminuirá la pérdida.

SGD tiene como ventajas que puede usar otras funciones de pérdida y que se puede paralelizar el proceso. Sin embargo tiene como desventajas que no converge rápidamente (es un algoritmo más lento que WALS) y es difícil incluir las entradas no observadas en la muestra. 

En contra posición, WALS tiene como desventaja que solo puede utilizar la perdida cuadrática, pero tiene como ventajas que puede ser paralelizado, converge más rápidamente que SGD y puede incluir las entradas no observadas en la muestra.

**Ventajas del filtrado colaborativo**
- No se necesita conocimiento del dominio porque las incrustaciones se aprenden automáticamente.
- El modelo puede ayudar a los usuarios a descubrir nuevos intereses. De forma aislada, es posible que el sistema de ML no sepa que el usuario está interesado en un elemento determinado, pero el modelo aún podría recomendarlo porque usuarios similares están interesados en ese elemento.
- Hasta cierto punto, el sistema solo necesita la matriz de retroalimentación para entrenar un modelo de factorización de matrices. En particular, el sistema no necesita características contextuales. En la práctica, esto se puede utilizar como uno de los múltiples generadores de candidatos.

**Desventajas del filtrado colaborativo**
- La predicción del modelo para un par dado (usuario, elemento) es el producto escalar de las incrustaciones correspondientes. Por lo tanto, si no se ve un elemento durante el entrenamiento, el sistema no puede crear una incrustación para él y no puede consultar el modelo con este elemento. Este problema a menudo se denomina problema de arranque en frío (*cold-start problem*). Sin embargo, las siguientes técnicas pueden abordar el problema del arranque en frío hasta cierto punto:
    - **Proyección en WALS:** Dado un elemento nuevo $i_0$ que no se ve en el entrenamiento, si el sistema tiene algunas interacciones con los usuarios, entonces el sistema puede calcular fácilmente una incrustación para este elemento sin tener que volver a entrenar todo el modelo. El sistema simplemente tiene que resolver la siguiente ecuación o la versión ponderada:
    $$\min_{v_{i_0}\in\mathbb{R}^d} ||A_{i_0} - U_{v_{i_0}}||$$
    
    La ecuación anterior corresponde a una iteración en WALS. Las incrustaciones de usuario se mantienen fijas y el sistema resuelve la incrustación de item $i_0$. Lo mismo se puede hacer para un nuevo usuario.

    - **Heurística para generar incrustaciones de elementos nuevos:** Si el sistema no tiene interacciones, el sistema puede aproximar su incrustación promediando las incrustaciones de elementos de la misma categoría, del mismo usuario que sube el contenido (en YouTube), etc.
    
- Difícil de incluir características secundarias para consulta/elemento. Las características secundarias son cualquier característica más allá de la consulta o el ID del elemento. Para recomendaciones de películas, las características secundarias pueden incluir el país o la edad. La inclusión de características laterales disponibles mejora la calidad del modelo. Aunque puede que no sea fácil incluir características secundarias en WALS, una generalización de WALS lo hace posible.

Para generalizar WALS, se aumenta la matriz de entrada con características definiendo una matriz de bloques $\bar{A}$, donde:

- El bloque (0,0) es la matriz de retroalimentación $A$ original.
- El bloque (0,1) es una codificación *one-hot* o de variables binarias de las características del usuario.
- El bloque (1,0) es una codificación *one-hot* o de variables binarias de las características del elemento.

# Referencias
- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022, de https://developers.google.com/machine-learning/recommendation/overview 